In [95]:
import xml.etree.ElementTree as ET
from lxml import etree
from datetime import datetime
import time

In [96]:
parser = etree.XMLParser(encoding="utf-8", recover=True)
f = open("xml/depart.xml", "r")
xmlstring = f.read()
f.close()
depart = etree.fromstring(xmlstring, parser=parser)
f = open("xml/arrive.xml", "r")
xmlstring = f.read()
f.close()
arrive = etree.fromstring(xmlstring, parser=parser)

In [131]:
logo_map = {
    'RusLine': 'Rusline.gif',
    'NordStar': 'Nordstar.gif',
    'British Airways': 'British.gif',
    'Aeroflot Russian Airlines': 'Aeroflot.gif',
    'Iberia': 'Iberia.gif',
    'Ural Airlines': 'Ural.gif',
    'Transaero': 'Transaero.gif',
    'Rossiya': 'Rossia.gif',
    'TAP-Air Portugal': 'Portugal.gif',
    'Siberia Airlines S7': 'S7.gif',
    'Ukraine International Airlines': 'Ukraine.gif',
    'Etihad Airways': 'Etihad.gif',
    'El Al': 'ElAl.gif',
    'Aegean Airlines': 'Aegean.gif',
    'Air France': 'France.gif',
    'KLM Royal Dutch Airlines': 'Klm.gif',
    'Qatar Airways': 'Qatar.gif',
    'Czech Airlines': 'Czech.gif',
    'International airport Orenburg': 'Orenburg.gif',
    'Delta Air Lines': 'Delta.gif',
    'Alitalia': 'Alitalia.gif',
    'UTair': 'Utair.gif',
    'Orenburg Airlines': 'Orenair.gif',
    'Gazpromavia Aviation Company': 'Gazprom.gif',
    'Yamal': 'Yamal.gif',
    'Belavia': 'Belavia.gif',
    '': 'Pobeda.gif',
}

In [170]:
records = []

for flight in depart.iter('flight'):
    if not 'dp_eng' in flight.attrib:
        continue
    date = datetime.strptime(flight.attrib['dp_eng'], "%d %b %H:%M")
    #print (date - datetime.strptime('23 Jul 13:41', "%d %b %H:%M")).total_seconds()
    to_push = {'flight': flight,
              'type': 'departure',
              'time': date}
    records.append(to_push)
    
for flight in arrive.iter('flight'):
    if not 'dp_eng' in flight.attrib:
        continue
    date = datetime.strptime(flight.attrib['dp_eng'], "%d %b %H:%M")
    #print (date - datetime.strptime('23 Jul 13:41', "%d %b %H:%M")).total_seconds()
    to_push = {'flight': flight,
              'type': 'arrival',
              'time': date}
    records.append(to_push)
    
records.sort(cmp= lambda x,y: int((x['time'] - y['time']).total_seconds()))

res = ''

head_html = '<!DOCTYPE html>\n\
<html>\n\
\t<head>\n\
\t\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n\
\t\t<link rel="stylesheet" type="text/css" href="css/table.css" />\n\
\t\t<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.5.2/jquery.min.js"></script>\n\
\t\t<script src="scripts/main.js"></script>\n\
\t</head>\n\
\t<body>\n\
\t\t<div class="jumbotron"><img src="img/a_back.png" /></div>\n\
\t\t<table class="buttons">\n\
\t\t\t<tr>\n\
\t\t\t\t<td id="arrive"><table><tr>\n\
\t\t\t\t\t<td><img src="img/glyph/g_arrival.png" class="flight-type" /></td>\n\
\t\t\t\t\t<td><h4>Arrival</h4></td>\n\
\t\t\t\t</tr></table></td>\n\
\t\t\t\t<td id="depart"><table><tr>\n\
\t\t\t\t\t<td><img src="img/glyph/g_departure.png" class="flight-type" /></td>\n\
\t\t\t\t\t<td><h4>Departure</h4></td>\n\
\t\t\t\t</tr></table></td>\n\
\t\t\t\t<td id="all" class="active"><h4>All</h4></td>\n\
\t\t\t</tr>\n\
\t\t</table>\n\
\t\t<h1>Timetable</h1>\n\
\t\t<div class="timetable">\n'
res += head_html

table = '\t\t\t<table id="main">\n\t\t\t\t<thead>\n'
table += '\t\t\t\t\t<tr><th></th><th class="logo"></th><th class="airline">Airline</th><th>Flight</th><th class="aircraft">Aircraft</th><th>Destination</th>'
table += '<th>Plan Time</th><th>Status</th><th class="note">Note</th></tr>\n'
table += '\t\t\t\t</thead>\n\t\t\t\t<tbody>\n'

for item in records:
    flight = item['flight']
    route = flight.find('route')
    process = flight.find('process')
    #print "{}/{}/{}".format(item['type'], flight.attrib['tws_arrive_eng'], flight.attrib['tws_depart_eng'])
    line = '<tr class="' + item['type'] + '">'
    
    # Type
    if item['type'] == 'departure':
        line += '<td><img src="img/glyph/g_departure.png" class="flight-type" /></td>'
    else:
        line += '<td><img src="img/glyph/g_arrival.png" class="flight-type" /></td>'
    # Logo
    
    if flight.attrib['m2_eng'] in logo_map:
        path = 'img/logo/' + logo_map[flight.attrib['m2_eng']]
    else:
        path = '#'
    line += '<td class="logo"><img src="' + path + '" class="logo" /></td>' 
    # Airline
    line += '<td class="airline">' + flight.attrib['m2_eng'] +'</td>'
    # Flight Number
    line += '<td class="id">' + flight.attrib['rf_eng'] + '-' + flight.attrib['flt'] + '</td>'
    # Airplane
    aircraft = ''
    if item['type'] == 'departure':
        line += '<td class="aircraft">' + flight.attrib['tws_depart_eng'] + '</td>'
        aircraft = flight.attrib['tws_depart_eng']
    else:
        line += '<td class="aircraft">' + flight.attrib['tws_arrive_eng'] + '</td>'
        aircraft = flight.attrib['tws_arrive_eng']
    # Airport
    line += '<td>' + flight.attrib['daname_eng'] + '</td>'
    # Time
    line += '<td>' + flight.attrib['dp_eng'] + '</td>'
    # Status
    line += '<td>' + flight.attrib['statuzz_eng'] + '</td>'
    # Note
    line += '<td class="note">' + flight.attrib['sovm_eng'] + '</td>'
    
    if item['type'] == 'departure':
        add_info = '<tr><td><span class="caption">Check-in</span><br />Begin:<br />End:<br />Check-in counters:<br />Status:</td>' +\
'<td><br />' + process.find('check-in').attrib['dt_b'] + '<br />' +\
process.find('check-in').attrib['dt_e'] + '<br />' +\
process.find('check-in').attrib['checkins_eng'] + '<br />' +\
process.find('check-in').attrib['status_eng'] + '</td></tr>'
        add_info += '<tr><td><span class="caption">Boarding</span><br />End:<br />Gates:<br />Status:</td>' +\
'<td><br />' + process.find('boarding').attrib['dt_e'] + '<br />' +\
process.find('boarding').attrib['gate_eng'] + '<br />' +\
process.find('boarding').attrib['status_eng'] + '</td></tr>'
    else:
        add_info = '<tr><td><span class="caption">Luggage</span><br />Status</td><td><br />' + process.find('baggage').attrib['status_eng'] + '</td></tr>'
        
    
    popLine = '<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />' +\
'<link rel="stylesheet" type="text/css" href="css/popup.css" /></head><body class="popup">' + \
'<h3 class="popup">' + flight.attrib['rf_eng'] + '-' + flight.attrib['flt'] + ' ' + flight.attrib['daname_eng'] + '</h3>' + \
'<div class="popup"><table class="popup"><tr><td>' + '<img src="' + path + '" class="logo" /></td>' + '<td><big>' + flight.attrib['m2_eng'] + '</big></td></tr>' + \
'<tr><td><span class="caption">Flight number</span><br />' + flight.attrib['rf_eng'] + '-' + flight.attrib['flt'] + '</td>' + \
'<td><span class="caption">Date</span><br />' + flight.attrib['dp_eng'] + '</td></tr>' + \
'<tr><td><span class="caption">AC Type ' + item['type'] + '</span><br />' + aircraft + '</td><td></td></tr>' + \
'<tr><td><span class="caption">Route</span><br />' + route.attrib['name_eng'].split(';')[0] + '<br />' + \
route.attrib['name_eng'].split(';')[1] + '</td><td><span class="caption">Status</span><br />' + route.attrib['status_eng'].split(';')[0] + \
'<br />' + route.attrib['status_eng'].split(';')[1] + '</td></tr>' + add_info + \
'</table></div></body></html>'
    
    line += '<td class="info">' + popLine + '</td></tr>'
    table += '\t\t\t\t\t' + line + '\n'

table += '\t\t\t\t</tbody>\n\t\t\t</table>'
res += table

tail_html = '\t\t</div>\n\t</body>\n</html>'
res += tail_html

f = open('index.html', 'w')
f.write(res)
f.close()